### Implement and test pytorch implementation of Multivariate Gaussian 

Generate some Gaussian 2D data and evaluate Gaussian in pytorch and scipy to test it. 

In [ ]:
import numpy as np 
import torch 
from torch.autograd import Variable
import scipy.stats

In [ ]:
# generate some 2D Gaussian samples with diverse means and covs 
d = 2
n = 10
means = np.random.randn(n, 2)
# generate random vectors of Us 
us = np.random.randn(n, d**2)
# cast them to matrices 
Us = us.reshape(n, d, d)
# make them triangular 
idx1, idx2 = np.tril_indices(d)
Us[:, idx1, idx2] = 0  # uups diagonal gone 
# set the diagonal again 
idx1, idx2 = np.diag_indices(d)
Us[:, idx1, idx2] = np.exp(np.random.randn(n, d))

# now we have the choleski transform of the precision matrix 
# get the covariance matrices  
S = np.zeros((n, d, d))

for idx in range(n): 
    S[idx, ] = np.linalg.inv(Us[idx,].T.dot(Us[idx,]))

In [ ]:
# sample the data 
data = []
models = []
for idx in range(n): 
    models.append(scipy.stats.multivariate_normal(mean=means[idx,], cov=S[idx,]))
    data.append(models[idx].rvs())
data = np.array(data)

In [ ]:
# now implement the pytorch version and compare the results 

# construct the pytorch Variables as batches 
X = Variable(torch.Tensor(data.tolist()))
mus = Variable(torch.Tensor(means.tolist()))
Us = Variable(torch.Tensor(Us.tolist()))

In [ ]:
def multivariate_normal_pdf(X, mus, Us, log=False):
    # dimension of the Gaussian 
    D = mus.size()[1]
    N = mus.size()[0]
    
    # get the precision matrices over batches using matrix multiplication: S^-1 = U'U
    Sin = torch.bmm(torch.transpose(Us, 1, 2), Us)
    
    norm_const = Variable(torch.zeros(N, 1))
    log_probs = Variable(torch.zeros(N, 1))
    
    for idx in range(N): 
        diagU = torch.diag(Us[idx, ])
        norm_const[idx] = (torch.sum(torch.log(diagU), -1) - (D / 2) * np.log(2 * np.pi)).unsqueeze(-1)

        diff = (X[idx, ] - mus[idx, ]).unsqueeze(-1)
        log_probs[idx] = - 0.5 * torch.mm(torch.transpose(diff, 0, 1), torch.mm(Sin[idx, ], diff))
        
    ps = norm_const + log_probs
    log_probs = ps 
    
    if log:
        return log_probs
    else: 
        return torch.exp(log_probs)

pdata_torch = multivariate_normal_pdf(X, mus, Us, log=True)

In [ ]:
# evaluate the scipy pdf as ground truth 
pdata_scipy = []
for idx in range(n): 
    pdata_scipy.append(np.log(models[idx].pdf(data[idx])))

In [ ]:
pdata_torch

In [ ]:
pdata_scipy